In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle


In [10]:
dataset = pd.read_csv("../data/raw/data.csv")
dataset.head()

,mora,atraso,vivienda,edad,dias_lab,exp_sf,nivel_ahorro,ingreso,linea_sf,deuda_sf,score,zona,clasif_sbs,nivel_educ
0,0,235,FAMILIAR,30,3748,93.0,5,3500.0,NaN,0.00,214,Lima,4,UNIVERSITARIA
1,0,18,FAMILIAR,32,4598,9.0,12,900.0,1824.67,1933.75,175,La Libertad,1,TECNICA
2,0,0,FAMILIAR,26,5148,8.0,2,2400.0,2797.38,188.29,187,Lima,0,UNIVERSITARIA
3,0,0,FAMILIAR,36,5179,20.0,12,2700.0,NaN,0.00,187,Ancash,0,TECNICA
4,0,0,FAMILIAR,46,3960,NaN,1,3100.0,2000.00,11010.65,189,Lima,0,TECNICA


In [23]:
# Variables que deseas eliminar de X
VARS_TO_DROP = ['mora']
CONTINUE_VARS_TO_IMPUT = ['edad', 'dias_lab', 'exp_sf', 'nivel_ahorro', 'ingreso', 'linea_sf', 'deuda_sf', 'score','clasif_sbs']
CATEGORICAL_VARS_TO_IMPUT = ['vivienda', 'zona', 'nivel_educ']
OHE_VAR_TO_ENCODE = ['vivienda', 'zona', 'nivel_educ']
# Definir X y y correctamente
x_features = dataset.drop(labels=VARS_TO_DROP, axis=1)  # Elimina las columnas no deseadas
y_target = dataset['mora']  # Selecciona la variable objetivo como una serie
# División de los datos
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_features, y_target, test_size=0.3, shuffle=True, random_state=2025)


In [24]:
from sklearn.pipeline import Pipeline
from feature_engine.imputation import MeanMedianImputer
from feature_engine.imputation import CategoricalImputer
from feature_engine.encoding import OneHotEncoder
from feature_engine.encoding import CountFrequencyEncoder
from sklearn.preprocessing import StandardScaler

In [31]:
create_features_and_base_pipeline = Pipeline([
    #Imputacion de variables continuas
    ('continues_var_mean_imputation',MeanMedianImputer(imputation_method='mean', variables=CONTINUE_VARS_TO_IMPUT)),
    #Imputacion de variables categoricas
    ('categorical_var_freq_imputation',CategoricalImputer(imputation_method='frequent',variables=CATEGORICAL_VARS_TO_IMPUT)),
    #Codificacion de variables categoricas
    ('categorical_encoding_ohe',OneHotEncoder(variables=OHE_VAR_TO_ENCODE, drop_last=True)),
    #estandarizacion de variables
    ('feature_scaling',StandardScaler())
 ])

In [32]:
#configuracion de pipeline ingeneria de caracteristicas
create_features_and_base_pipeline.fit(X_train)

Pipeline(steps=[('continues_var_mean_imputation',
                 MeanMedianImputer(imputation_method='mean',
                                   variables=['edad', 'dias_lab', 'exp_sf',
                                              'nivel_ahorro', 'ingreso',
                                              'linea_sf', 'deuda_sf', 'score',
                                              'clasif_sbs'])),
                ('categorical_var_freq_imputation',
                 CategoricalImputer(imputation_method='frequent',
                                    variables=['vivienda', 'zona',
                                               'nivel_educ'])),
                ('categorical_encoding_ohe',
                 OneHotEncoder(drop_last=True,
                               variables=['vivienda', 'zona', 'nivel_educ'])),
                ('feature_scaling', StandardScaler())])

In [39]:
# Aplicar transformaciones con el pipeline
x_features_processed = create_features_and_base_pipeline.fit_transform(X_train)
# Obtener los nombres de las columnas después del pipeline
processed_columns = create_features_and_base_pipeline.named_steps['categorical_encoding_ohe'].get_feature_names_out()
# Crear un DataFrame con las columnas procesadas
df_features_process = pd.DataFrame(x_features_processed, columns=processed_columns)
df_features_process['mora'] = y_train
df_features_process['mora'].fillna(1,inplace=True)
# Verificar el resultado
df_features_process.head()

C:\Users\karen\AppData\Local\Temp\ipykernel_12328\4028471846.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_features_process['mora'].fillna(1,inplace=True)


,atraso,edad,dias_lab,exp_sf,nivel_ahorro,ingreso,linea_sf,deuda_sf,score,clasif_sbs,...,zona_Huanuco,zona_Tumbes,zona_Apurimac,zona_Tacna,zona_Madre de Dios,zona_Huancavelica,nivel_educ_TECNICA,nivel_educ_UNIVERSITARIA,nivel_educ_SECUNDARIA,mora
0,1.451668,-0.746931,-0.015363,-5.270941e-01,0.419561,-0.685357,-0.523399,-0.533502,-0.122702,-0.618452,...,-0.06665,-0.050576,-0.031963,-0.053852,-0.022595,-0.018447,1.248324,-1.151327,-0.185712,0.0
1,-0.319558,-0.340789,1.189311,-2.434089e-16,0.419561,-0.581198,-0.562019,-0.318024,-0.477133,0.501659,...,-0.06665,-0.050576,-0.031963,-0.053852,-0.022595,-0.018447,-0.801074,0.868563,-0.185712,1.0
2,-0.319558,-0.645396,0.593491,9.459467e-01,0.419561,-0.155696,-0.413876,-0.325967,1.143121,0.501659,...,-0.06665,-0.050576,-0.031963,-0.053852,-0.022595,-0.018447,-0.801074,0.868563,-0.185712,0.0
3,-0.319558,-0.645396,-0.542757,-8.011482e-01,-3.129927,-0.083730,-0.267015,-0.557250,-1.084728,-0.618452,...,-0.06665,-0.050576,-0.031963,-0.053852,-0.022595,-0.018447,-0.801074,0.868563,-0.185712,0.0
4,-0.319558,0.166889,-0.270914,-8.011482e-01,0.419561,-0.513145,-0.498164,-0.064453,0.079829,-0.618452,...,-0.06665,-0.050576,-0.031963,-0.053852,-0.022595,-0.018447,1.248324,-1.151327,-0.185712,1.0


In [36]:
create_features_and_base_pipeline.transform(X_test)

array([[-0.24575663,  1.48685084, -0.25648369, ...,  1.24832351,
        -1.15132697, -0.18571168],
       [-0.24575663, -1.25460919, -0.76665467, ..., -0.8010744 ,
         0.86856299, -0.18571168],
       [-0.31955769,  0.26842416,  2.49220213, ..., -0.8010744 ,
         0.86856299, -0.18571168],
       ...,
       [-0.17195558, -0.03618251, -0.28441276, ..., -0.8010744 ,
         0.86856299, -0.18571168],
       [-0.31955769,  0.57303083, -0.80854827, ..., -0.8010744 ,
         0.86856299, -0.18571168],
       [-0.31955769,  1.18224417,  1.14648652, ..., -0.8010744 ,
         0.86856299, -0.18571168]])

In [46]:
#guardamos los datos para el modelo
df_features_process.to_csv('../data/processed/features_for_model.csv')

with open('../artifacts/pipeline.pkl','wb') as f:
    pickle.dump(create_features_and_base_pipeline, f)